In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 

from wordcloud import WordCloud
import matplotlib.pyplot as plt 


In [2]:

url="https://www.imdb.com/"
response=requests.get(url)
soup=BeautifulSoup(response.text,'html')
soup

<html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
</body>
</html>

In [3]:
soup.find_all('table')[0]

IndexError: list index out of range